In [1]:
## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os, alive_progress
from alive_progress import alive_bar

from itertools import combinations
from scipy.stats import mannwhitneyu as mwu
from utils import VD_A as vda
from utils import apfd
from utils import cartesian
from utils import derive_data
from utils import sort_vda
from utils import calc_s12

def _interp_addsorted(alist, datapoints=[]):
    cc_dp = alist.copy()
    for newdp in datapoints:
        if(newdp in cc_dp): continue
        cc_dp = np.insert(cc_dp,np.searchsorted(cc_dp,newdp),newdp)
    return cc_dp

def interp(data: pd.DataFrame.dtypes, col_costs: str, col_hypsizes: str, datapoints=[]):
    df_subset = data.copy()
    df_subset[col_hypsizes+'_withdatapoints']=df_subset[col_hypsizes].apply(lambda x: _interp_addsorted(x,datapoints))
    df_subset[col_costs]=df_subset.apply(lambda x: np.interp(x[col_hypsizes+'_withdatapoints'], x[col_hypsizes], x[col_costs]),axis=1)
    df_subset[col_hypsizes]=df_subset[col_hypsizes+'_withdatapoints']
    df_subset.drop(col_hypsizes+'_withdatapoints',inplace=True,axis=1)
    return df_subset


# set inline print
%matplotlib inline

os.makedirs(f'data/img/', exist_ok=True)

# Load CSV, calculate APFD and filter Equivalent==OK

In [2]:
df_fixed = pd.concat([pd.read_csv(os.path.join(pth)) for pth in ["k_1_k_2_hads_hsi_w_wp_h_fixed.csv", "k_1_k_2_spy_spyh.csv"]])
df_fixed = derive_data(df_fixed)
df_fixed.EquivalenceOracle=df_fixed.CTT
df_fixed['Type'] = 'Fixed'

df_random = pd.read_csv(os.path.join("random_logs.csv"))
df_random = derive_data(df_random)
df_random.EquivalenceOracle=df_random.apply(lambda x: x.CTT+','+str(x['Random Infix Length']), axis=1)
df_random['Type'] = 'Random'
df_random['EquivalenceOracle'] = pd.Categorical(df_random['EquivalenceOracle'], 
  [f'{x[0]},{x[1]}' for x in list(cartesian((["HadsInt", "Hsi", "Wp", "W"], df_random['Random Infix Length'].unique()))
)])

# Concatenate dataframes and derive AFPD

In [3]:
# equivs_drv = pd.concat([df_fixed])
equivs_drv = pd.concat([df_fixed,df_random])
equivs_drv = pd.merge(equivs_drv, pd.read_csv(os.path.join("SUL_list.csv")), how='left',on='SUL name')
equivs_drv = equivs_drv.query(f'`Equivalent`=="OK" and `Extra States`==2')

# Plot %states detected per test case (for all methods)

In [4]:
# define figure DPI
sns.set(rc={'figure.dpi':400})

all_runs = equivs_drv[['SUL name']].drop_duplicates()
# all_runs = equivs_drv.query(f'`SUL name`in ["TCP_Linux_Client.dot"]')[['SUL name']].drop_duplicates()
total = len(all_runs)

new_dps = list(range(10,101,10))
new_dps.reverse()

# methods = ["W", "Wp", "SPY", "HadsInt"]
methods = ["W", "Wp", "Hsi", "H", "SPY", "SPYH", "HadsInt"]
with alive_bar(total, force_tty=True, title=f'Plotting APFD') as bar:
    for idx,row in all_runs.iterrows():
        # get an entry <SUL, seed>
        sulname = row['SUL name']
        fname=sulname.replace('.dot','')
        subj=equivs_drv.query(f'`SUL name`=="{sulname}" and `Type`=="Fixed" and `CTT` in @methods').copy()
        
        apfd_tab = subj[['CTT','MQ','EQ','TC','Rounds','APFD']].sort_values(['APFD'],ascending=False)
        apfd_tab.to_csv(f'data/img/{fname}.csv')
        
        # add percent columns
        subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)

        # analyze qtype column 
        qtype = 'Total cost'
        
        subj_interp=interp(data=subj, col_costs=qtype,col_hypsizes='HypSizePercent', datapoints=new_dps)
        subj_interp=subj_interp.explode(['HypSizePercent',qtype])
        subj_interp=subj_interp.query('HypSizePercent in @new_dps').sort_values(by=[qtype],ascending=False)

        cols_order = subj_interp['CTT'].drop_duplicates().to_list()
        cols_order.reverse()

        the_cols = ['SUL name','HypSizePercent','CTT']
        subj_gb=subj_interp[[*the_cols,qtype]].groupby(the_cols).first()

        subj_pvt = subj_gb.pivot_table(qtype,['CTT'],'HypSizePercent').sort_values(by=list(new_dps))
        #subj_pvt = subj_pvt/subj_pvt.max().max()
        heatmap_cost=sns.heatmap(subj_pvt,
                         annot=True, fmt=",.0f", annot_kws={'rotation': 0, 'fontsize':9}, 
                         cmap= sns.cm.rocket_r, linewidth=.5, linecolor='black'
                        )

        #add plot labels, titles and legends
        plt.xlabel('Fraction of the SUL learned')
        plt.ylabel('')
        #plt.legend(title='Testing Technique', loc='lower right')
        plt.title(f'Interpolation of {qtype}')

        # save line chart
        fig = heatmap_cost.get_figure()
        plt.gcf().set_size_inches(30,20)
        fig.savefig(f'data/img/{fname}_heatmap.pdf', bbox_inches='tight')
        fig.clf()
        
        # explode qtype column with % of cost and hypothesis sizes in the learning process
        subj=subj.explode(['HypSizePercent',qtype])
        subj[[*the_cols,qtype]].to_csv(f'data/img/{fname}_HypSizePercent.csv')

        #create line chart
        apfd_plot = sns.lineplot(subj, x=f'{qtype}', y='HypSizePercent',
                                 markers=True, 
                                 style='CTT', hue='CTT',
                                 palette='tab10'
                                )
        apfd_plot.set(xscale='linear', yscale='linear')
        apfd_plot.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True,useOffset=True, useLocale=True))
        apfd_plot.yaxis.set_major_formatter(ticker.PercentFormatter())
        
        #add plot labels, titles and legends
        plt.xlabel(f'{qtype.title()}')
        plt.ylabel('')
        _sulname=sulname.replace('.dot','')
        plt.title(f'Subject: {_sulname}\nFraction of the SUL learned vs. {qtype}')

        #get handles and labels
        handles, labels = plt.gca().get_legend_handles_labels()

        #specify order of items in legend from APFD
        order = subj[['CTT','APFD']].sort_values(['APFD'],ascending=False).drop_duplicates().CTT.to_list()
        for idx,ctt in enumerate(order): order[idx]=labels.index(ctt)
        #add legend to plot
        plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                   title='Testing Technique', loc='lower right', 
                   fontsize='xx-small', title_fontsize='xx-small')

        # save line chart
        fig = apfd_plot.get_figure()
        plt.gcf().set_size_inches(8,5)
        fig.savefig(f'data/img/{fname}.pdf', bbox_inches='tight')
        fig.clf()
        
        bar()

Plotting APFD |████████████████████████████████████████| 46/46 [100%] in 1:08.2 


<Figure size 3200x2000 with 0 Axes>

# Draw boxplots for TC and APFD

In [5]:
dict_of_metrics = {'s1':['TC_s1', 'APFD_s1'], 'normalized':['TC_s2', 'APFD_s1']}

for ctt_mode in equivs_drv.Type.unique():
    dataset = equivs_drv.query(f'`Type`=="{ctt_mode}"')
    metrics_s12 = calc_s12(dataset).reset_index()
    for metric_name,metrics_to_plot in dict_of_metrics.items():
        # initialize figure with 2 subplots in a row
        fig, ax = plt.subplots(1, 2, figsize=(12,3), dpi=400)

        # add padding between the subplots
        plt.subplots_adjust(wspace=0.2,hspace=0.3)

        idx_inc=0
        for metric in metrics_to_plot:
            idx=idx_inc
            # draw plots
            sns.boxplot(data=metrics_s12, ax=ax[idx], x='EquivalenceOracle',y=metric)
            ax[idx].set_xlabel('')
            ax[idx].set_ylabel(metric.replace(f'_{metric_name}',''))
            #ax[idx].set_xlim([0,1])
            ax[idx].tick_params(axis='x', rotation=0, labelsize=8)
            if "_s1" in metric and not "APFD_" in metric: ax[idx].set(yscale='log')
            idx_inc=idx_inc+1
        ##add overall title to replot
        #fig.suptitle(f'{ctt_mode} mode')
        
        # save line chart
        fig.savefig(f'data/{ctt_mode}_{metric_name}.pdf', bbox_inches='tight')
        fig.clf()


<Figure size 4800x1200 with 0 Axes>

<Figure size 4800x1200 with 0 Axes>

<Figure size 4800x1200 with 0 Axes>

<Figure size 4800x1200 with 0 Axes>

# Write tables for TC and APFD

In [6]:
for name in equivs_drv.Type.unique():
    dataset = equivs_drv.query(f'`Type`=="{name}"')\
                .sort_values(['SUL name','EquivalenceOracle','Seed'])\
                .copy()
    metrics_s12 = calc_s12(dataset).reset_index()
    for metric_name,metrics_to_plot in dict_of_metrics.items():
        for metric in metrics_to_plot:
            results=metrics_s12.sort_values(['SUL name','EquivalenceOracle'])\
                        .groupby(['EquivalenceOracle'])\
                        .apply(lambda x: x[metric].tolist())\
                        .reset_index().sort_values(['EquivalenceOracle'],ascending=False)
            results=results.reindex(index=results.index[::-1])
            # apply combination method
            results = dict(zip(list(combinations(results['EquivalenceOracle'], 2)),list(combinations(results[0], 2))))
            results = pd.DataFrame.from_dict(results, orient='index').reset_index()
            results['mwu'] = results.apply(lambda x: mwu(x[0],x[1]).pvalue,axis=1)
            results['mwu<0.05'] = results['mwu']<0.05
            results[['vda_estimate','vda_magnitude']] = results.apply(lambda x: pd.Series(vda(x[0],x[1])),axis=1)
            results[['A','B']] = results['index'].apply(lambda x: pd.Series([x[0],x[1]]))
            results.drop([0,1,'index'],axis=1,inplace=True)
            results.set_index(['A','B'],inplace=True)
            results.columns.name = f'{metric} ({name})'
            results.to_csv(f'data/{metric}_{name}.csv', float_format='%.4f')
            results.style.to_latex(f'data/{metric}_{name}.tex', column_format='%.4f')